In [101]:
import sys
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

sys.path.append('pyttr')
from pyttr.ttrtypes import BType, PType, Possibility, Pred, RecType, Rec, FunType, Fun

## ResNet50 classification

In [49]:
model = ResNet50()

In [50]:
def classify(path):
    img = load_img(path, target_size=(224,224))
    input = preprocess_input(np.array([img_to_array(img)]))
    prediction = decode_predictions(model.predict(input))[0]
    return prediction[0][1].replace('_', ' ')
classify('res/canal.jpg')

'palace'

In [51]:
M = Possibility('M1')
Ind = BType('Ind')
Ind.in_poss(M)

def see(path):
    pred = Pred(classify(path), [Ind])
    name = path.split('/')[-1].split('.')[0]
    Ind.judge(name)
    PType(pred, [name]).in_poss(M).judge(path)

In [52]:
see('res/volcano.jpg')
see('res/volcano2.jpg')
see('res/canal.jpg')
see('res/dogcar.jpg')
#see('/home/arildm/Pictures/star.png')
#see('/home/arildm/Pictures/skocaj.png')
print(M.show())


M1:
_____________________________________________
Ind: [volcano, volcano2, canal, dogcar]
palace(canal): [res/canal.jpg]
volcano(volcano2): [res/volcano2.jpg]
volcano(volcano): [res/volcano.jpg]
Norfolk terrier(dogcar): [res/dogcar.jpg]
_____________________________________________



## Classification as a witness condition

In [53]:
words = ['volcano', 'palace', 'dog', 'canal']
ptypes = []
for word in words:
    ptype = PType(Pred(word, [Ind]), ['x'])
    ptype.learn_witness_condition(lambda path, word=word: classify(path) == word)
    ptypes.append(ptype)
for path in ['res/volcano2.jpg', 'res/canal.jpg', 'res/dogcar.jpg']:
    for ptype in ptypes:
        if ptype.query(path):
            print(path + ' is a ' + ptype.show())

res/volcano2.jpg is a volcano(x)
res/canal.jpg is a palace(x)


### Questionable "object detector" function

Still with ResNet50, meaning only one class for the image, no object detection.

In [70]:
imageT = BType('Image')

def object_detector_simple(path):
    det = set()
    for ptype in ptypes:
        if ptype.query(path):
            det.add(RecType({
                'reg': str(load_img(path)),
                'pfun': Fun('x', imageT, ptype),
            }))
    return det
            
print([t.show() for t in odf('res/volcano2.jpg')])

['{pfun : lambda x:Image . volcano(x), reg : <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1200x900 at 0x7FCB0B03CEF0>}']
